In [1]:
#Importing the required modules
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Read and Show the input image 
original_image=cv2.imread('otherangle2.jpeg')
cv2.imshow('Original_image',original_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
#For highlighting a specific region og the image and supressing the other regions
def region_of_interest(image):
    img_height = image.shape[0]
    img_width = image.shape[1]
    # Define the vertices of a polygon that represents the region of interest (ROI).
    # Here we highlight the BOTTOM part of the image and maskout the remaining part of the image
    vertices = np.array([
        [(0,img_height),
         (0, img_height*0.3),
         (img_width, img_height*0.3),
         (img_width, img_height)]
    ], dtype=np.int32)
    
    # Create an empty mask image with the same dimensions as the input image.
    mask= np.zeros_like(image)
    # Fill the region defined by the polygon with a white color (255) to create the mask.
    cv2.fillPoly(mask, vertices, 255)
    # Bitwise AND between the input image and the mask
    # This operation retains the pixel values in the ROI (defined by the mask) and sets the rest to zero.
    highlighted_image = cv2.bitwise_and(image, mask)
    
    return highlighted_image
    

In [4]:
#Detecting edges from the input image

def canny(image):
    # Convert to grayscale as only image intensity needed for gradients
    gray_image= cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    
    # 5x5 gaussian blur to reduce noise 
    blur = cv2.GaussianBlur(gray_image, (5,5), 0)
    
    # Canny edge detector with minVal of 50 and maxVal of 150
    edges = cv2.Canny(blur, 70, 180)
    return edges


In [5]:
#For correcting the distortions present in the fish eye input image
def distortion_correction(image):
    #Camera orientation and Field Of View Parameters in degrees
    cameraHeading= 90.0 #cameras horizontal direction or rotation
    cameraPitch= 90.0 #cameras vertical direction or rotation
    cameraFOV= 90.0 #cameras field of view in degrees
    #dimensions of input image
    img_width, img_height = original_image.shape[1],original_image.shape[0]
    print(f"Input Image width = {img_width}")
    print(f"Input Image height = {img_height}")

    #dimensions of destination image that will hold the corrected image
    outimg_width= img_width-20
    outimg_height= img_height-20

    #Initializing camera parameters
    DEG2RAD= math.pi/180.0 #converting degree to radians

    #Calculating the required camera parameters
    vertical_map= img_height / math.pi # for mapping cameras vertical angles
    horizontal_map= 0.5*img_width / math.pi #for mapping the entire range of horizontal angles
    upFactor= 2.0 * math.tan(cameraFOV* DEG2RAD / 2.0) #factor to control amount of vertical stretching or compression
    rightFactor= 1.33 * upFactor #factor to achieve desired horizontal correction

    #calculating direction vector of camera
    X_cameraDir= math.sin(cameraPitch * DEG2RAD) * math.sin(cameraHeading * DEG2RAD)
    Y_cameraDir= math.cos(cameraPitch * DEG2RAD)
    Z_cameraDir= math.sin(cameraPitch * DEG2RAD) * math.cos(cameraHeading * DEG2RAD)

    #calculating up vector of camera using upFactor
    X_cameraUp= upFactor * math.sin((cameraPitch-90.0)*DEG2RAD)* math.sin(cameraHeading* DEG2RAD)
    Y_cameraUp= upFactor * math.cos((cameraPitch-90.0)*DEG2RAD)
    Z_cameraUp= upFactor * math.sin((cameraPitch-90.0)*DEG2RAD)* math.cos(cameraHeading* DEG2RAD)

    #calculating right vector of camera using rightFactor
    X_cameraRight= rightFactor * math.sin((cameraHeading-90.0)*DEG2RAD)
    Y_cameraRight= 0.0
    Z_cameraRight= rightFactor * math.cos((cameraHeading-90.0)*DEG2RAD)

    #calculating origin of the camera plane
    X_cameraPlaneOrigin= X_cameraDir + 0.5 * X_cameraUp - 0.5 * X_cameraRight
    Y_cameraPlaneOrigin= Y_cameraDir + 0.5 * Y_cameraUp - 0.5 * Y_cameraRight
    Z_cameraPlaneOrigin= Z_cameraDir + 0.5 * Z_cameraUp - 0.5 * Z_cameraRight
    FOV= math.pi * 130.0 / 180.0
    
    #Creating an empty destination image to hold the corrected image
    size= outimg_height, outimg_width, 3
    outimg= np.zeros(size, dtype=float)
    
    #for accessing each individual pixel in output image space
    for i in range(1, outimg_height):
        for j in range(1, outimg_width):
            #fractional position of current pixel in output image space
            fx= float(j) / float(outimg_width)
            fy= float(i) / float(outimg_height)
            
            #computing the 3D ray direction for the current pixel
            X_ray= X_cameraPlaneOrigin + fx* X_cameraRight - fy* X_cameraUp
            Y_ray= Y_cameraPlaneOrigin + fx* Y_cameraRight - fy* Y_cameraUp
            Z_ray= Z_cameraPlaneOrigin + fx* Z_cameraRight - fy* Z_cameraUp
            #Calculating magnitutde of the ray direction
            normRay = math.sqrt(Y_ray*Y_ray + Z_ray*Z_ray)
            
            #Calculating fisheye angle and radius
            azimuthal_angle = math.atan2(Z_ray, Y_ray) #angle measured in xy plan from x axis towards y axis
            polar_angle = math.atan2(normRay, X_ray) #angle measured from z axis towards xy plane
            # represents the radial distance in the fisheye image from the image center to the current pixel
            # value of r indicates how far a pixel should be from the image center to correct the distortion
            #controls how much each pixel should be moved from its distorted position to its corrected position
            radius= img_height * polar_angle / FOV 
            
            #Mapping to pixels between distorted and corrected image
            
            #represents the x-coordinate (column) in the corrected fisheye image where the current pixel from the original fisheye image should be placed
            azimuthal_map = math.floor(0.5*img_width + radius * math.sin(azimuthal_angle))
            #represents the y-coordinate (row) in the corrected fisheye image where the current pixel from the original fisheye image should be placed
            polar_map = math.floor(0.5*img_height - radius*math.cos(azimuthal_angle))
            outimg_offset = (i*outimg_width + j) #calculates an index corresponding to the position of the pixel in the corrected image. Used for accessing and set pixel values in the corrected image
            img_offset = (polar_map * img_width + azimuthal_map) #calculates index corresponding to the position of the pixel in the distorted fisheye image
            row = max(min(int(polar_map),img_height-1),1) #represents row_index
            #bounded by 1 to to avoid negative indices and img_height ensures that the indices stay within image bounds
            col = max(min(int(azimuthal_map),img_width-1),1) #represents co_index
            
            #Incorporating colour values from input image to output image by obtaining corrected colour values for current pixels via INTERPOLATION
            outimg[i,j,0]=image[row,col,0]/255.0 #divided by 255.0 to scale the values within an range of [0,1]
            outimg[i,j,1]=image[row,col,1]/255.0
            outimg[i,j,2]=image[row,col,2]/255.0
            
    return outimg * 255
            

In [6]:
#Displaying Input Image
cv2.imshow('Input Image',original_image)

#Correcting distorted image
corrected_image=distortion_correction(original_image)
corrected_image = corrected_image.astype(np.uint8)
height,width=corrected_image.shape[0],corrected_image.shape[1]
print(f'img_height= {height}')
print(f'image_width = {width}')

#Displaying Corrected Image
cv2.imshow("corrected fisheyee image",corrected_image)
cv2.imwrite('corrected fisheyee otherangle2.jpg',corrected_image)

#Detecting and Displaying edges
canny_edges = canny(corrected_image)
cv2.imshow('canny edges',canny_edges)
cv2.imwrite('canny edges otherangle2.jpg',canny_edges)

#Highlighting only a specific region of image
masked_image=region_of_interest(canny_edges)
cv2.imshow('canny edges after applying mask',masked_image)
cv2.imwrite('roi mask otherangle2.jpg',masked_image)

#Performin Hough lines to detect the lanes
lines = cv2.HoughLinesP(
    masked_image,
    rho=2,
    theta=np.pi / 180,
    threshold=80,
    lines=np.array([]),
    minLineLength=80,
    maxLineGap=5
)

#Drawing the detected lanes onto the image
for line in lines:
    x1,y1,x2,y2=line[0]
    cv2.line(corrected_image,(x1,y1),(x2,y2),(0,255,0),2)

# Display the corrected image with lane detection
cv2.imshow("Lane Detection", corrected_image)
cv2.imwrite('lane_detection_otherangle2.jpg',corrected_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
            

Input Image width = 320
Input Image height = 158
img_height= 138
image_width = 300
